# Minimum Complementary Energy

Over the past two weeks, we have explored the displacement resulting from distributed pressure and the Hertz contact solution. Now, we shift our focus to real contact problems, emphasizing **Variational Principles** and the **Minimum Complementary Energy** approach.

![Contact model discussed in first step](figures/Problem_mapping.png)

Here we simplify our model as the contact between a rigid rough surface and an elastic flat surface[4].

![rigid rough surface elastic contact](figures/rigid_rough_surface_elastic_contact.jpg)


We want to know pressure distribution $P(x,y)$, and we define our gap function as:

$$
g(x, y)=u_z(x, y)-h(x, y)
$$

where $u_z(x, y)$ is the normal displacement, the separation of two surfaces $h(x,y)$ can sometimes be $h(r)=-\frac{r^2}{2 R}$, like Rey(2017)[5].

As the first chapiter of Lucas(2020)[1], the potential energy can be derived from elastic energy relationship $\frac{1}{2} \int_v \varepsilon: \mathbb{C}: \varepsilon d v$ to:

$$
E_p\left(u_z, p\right)=\frac{1}{2} \int_s u_z \cdot p d s
$$

If we minimize this potential energy with respect to $u_z$, then we call it **Primal minimization problem**, we will end up with a displacement solution.

$$
\min _{g\left[u_z\right] \geqslant 0} E_p\left(u_z, p\left[u_z\right]\right)
$$

Here we focus on complementary energy minimization, called **Dual minimization problem**, where complementary energy is defined as:

$$
E_c\left(u_z, p\right)=\frac{1}{2} \int_s u_z \cdot p d s-\int_s p h d s
$$

Then the minimization could be:

$$
\min _{p \geqslant 0} E_c\left(u_z[p], p\right)
$$

It is obvious that we will end up with a pressure solution in this procedure.



## Q1: Legendre transform for dual optimization?

From what we discuss above, we have two constraints, 

$$
\frac{1}{S} \int_s p d s=\bar{p}
$$

And

$$
p \geq 0
$$

To minimize the complementary energy, we consider:

$$
\lim _{\varepsilon \rightarrow 0} \frac{E_c(p+\varepsilon v)-E_c(p)}{\varepsilon}
$$

Here we can derive a gap function by:

$$
\frac{\partial E_c}{\partial p}=u_z[p]-h=g
$$



We choose the same background as a start point to verify this energy minimization, and define $ \bar{p}=\frac{W}{S} $

### *$u_z$* is our unknown in this case, but we know the relationship between *$u_z$* and pressure *$P$*, we will then treat *$u_z$* as *$u_z(P)$*. The Hertz solution is derived from elastic theory, here we want to derive from energy theory, finally we can compare with Hertz solution.

#  Complementary energy minimization for Normal contact

We follow the steps of Lucas(2020)[1], and finally we will have two alrgorithms to implement, Steepest descent[6] and Constrained conjugate gradient[7]. First we give the notation as reference:

Let the open $\mathcal{B} \subset \mathbb{R}^3$ be a deformable elastic solid of boundary $\partial \mathcal{B}$ with outward normal $\boldsymbol{n}$. Let $u$ be the displacement vector field of $\mathcal{B}$.

We have the following strain-stress relationship in the context of continuum mechanics and elasticity theory:

$$
\boldsymbol{\varepsilon}[\boldsymbol{u}]:=\frac{1}{2}\left(\boldsymbol{\nabla} \boldsymbol{u}+\boldsymbol{\nabla} \boldsymbol{u}^T\right)
$$
$$
\sigma[u]:=C: \varepsilon[u]
$$

We define the traction vector of the displacement field $\boldsymbol{u}$ as:

$$
T[u]:=\left.\sigma[u]\right|_{\partial \mathcal{B}} \cdot \boldsymbol{n}
$$

$\boldsymbol{b}$ is given as the volumetric force density, then with given boundary condition, we can express the principle of minimum potential energy:

$$
\inf _{\boldsymbol{u} \in \mathrm{KA}(\mathcal{B})}\left\{I(\boldsymbol{u})=\frac{1}{2} \int_{\mathcal{B}} \boldsymbol{\sigma}[\boldsymbol{u}]: \boldsymbol{\varepsilon}[\boldsymbol{u}] \mathrm{d} V-\int_{\mathcal{B}} \boldsymbol{b} \cdot \boldsymbol{u} \mathrm{d} V-\int_{\partial \mathcal{B}} T[\boldsymbol{u}] \cdot \boldsymbol{u} \mathrm{d} S\right\}
$$

which involves finding $u \in \mathrm{KA}(\mathcal{B})$, i.e. a kinematically admissible displacement field, that minimizes the total potential energy of the system.






As we mentioned before, we want to minimize complementary energy for a non-friction, non-adhesion contact between the boundary $\partial \mathcal{B}$ and a surface $\mathcal{S}$, we can introduce the gap function as:

$$
g[\boldsymbol{u}]:=\left.\boldsymbol{u} \cdot \boldsymbol{e}_3\right|_{\partial \mathcal{B}}-h
$$

The boundary conditions on $\partial \mathscr{B}$ are hence expressed with the Hertz-Signorini-Moreau conditions:

$$
\begin{aligned}
g[\boldsymbol{u}] & \geq 0 \\
p[\boldsymbol{u}]:=\boldsymbol{T}[\boldsymbol{u}] \cdot \boldsymbol{e}_3 & \geq 0 \\
g[\boldsymbol{u}] p[\boldsymbol{u}] & =0
\end{aligned}
$$

The contact condition that we want to apply for **Steepest descent** algorithm as stopping criteria is:

$$
g[\boldsymbol{u}] p[\boldsymbol{u}]=0
$$



### Variational form

For this contact problem, we have two constraints, 

$$
\frac{1}{S} \int_s p d s=\bar{p}
$$

And

$$
p \geq 0
$$

To introduce the contact constraints, one needs to define the space of kinematically admissible displacement fields $\mathrm{KA}(\mathcal{B})$. The first consideration defines the function space

$$
\bar{H}^1\left(\mathcal{B}_p ; \mathbb{R}^3\right)=\left\{\boldsymbol{u}+\overline{\boldsymbol{u}} \mid \boldsymbol{u} \in H^1\left(\mathcal{B}_p ; \mathbb{R}^3\right), \overline{\boldsymbol{u}} \in \mathbb{R}^3 \text { with } \widehat{\mathbf{u}}(\mathbf{0}, \bullet)=0\right\}
$$

which is a space of $\mathcal{B}_p$-periodic functions whose fundamental mode (i.e. the horizontal average) is constant with respect to $x_3$. The space of admissible solutions to the contact problem is:

$$
\mathrm{KA}\left(\mathcal{B}_p\right):=\left\{\boldsymbol{u} \in \bar{H}^1\left(\mathcal{B}_p ; \mathbb{R}^3\right): g[\boldsymbol{u}] \geq 0\right\}
$$

The minimization principle is then written as:

$$
\inf _{\boldsymbol{u} \in \operatorname{KA}\left(\mathcal{B}_p\right)}\left\{\frac{1}{2} \int_{\mathcal{B}_p} \boldsymbol{\sigma}[\boldsymbol{u}]: \boldsymbol{\varepsilon}[\boldsymbol{u}] \mathrm{d} V\right\}
$$

which is a quadratic program with unilateral constraints. Kalker(1977)[3] has shown that the above problem can be expressed only in terms of boundary integrals, provided $\boldsymbol{u}$ additionally satisfies the local equilibrium(strong Euler-Lagrange equation) $\operatorname{div}(\sigma[\boldsymbol{u}])+\boldsymbol{b}=\mathbf{0} \quad \text { a.e. in } \mathcal{B}$:

$$
\begin{aligned}
\frac{1}{2} \int_{\mathcal{B}_p} \sigma[\boldsymbol{u}]: \boldsymbol{\varepsilon}[\boldsymbol{u}] \mathrm{d} V & =\frac{1}{2} \int_{\mathcal{B}_p} \boldsymbol{\sigma}[\boldsymbol{u}]: \boldsymbol{\nabla} \boldsymbol{u} \mathrm{d} V \\
& =\frac{1}{2} \int_{\mathcal{B}_p}(\operatorname{div}(\sigma[\boldsymbol{u}] \cdot \boldsymbol{u})-\operatorname{div}(\sigma[\boldsymbol{u}]) \cdot \boldsymbol{u}) \mathrm{d} V \\
& =\frac{1}{2} \int_{\partial \mathcal{B}_p} \boldsymbol{u} \cdot \boldsymbol{\sigma}[\boldsymbol{u}] \cdot \boldsymbol{n} \mathrm{d} V
\end{aligned}
$$

so that the contact problem is equivalent to

$$
\inf _{\boldsymbol{u} \in \operatorname{KA}\left(\mathcal{B}_p\right)}\left\{I_u(\boldsymbol{u})=\frac{1}{2} \int_{\partial \mathcal{B}_p} \boldsymbol{T}[\boldsymbol{u}] \cdot \boldsymbol{u} \mathrm{d} S\right\}
$$

This minimization problem can now be solved by quadratic programming (see Rey(2017)[5]), but optimization techniques are usually applied to the dual of $\inf _{\boldsymbol{u} \in \operatorname{KA}\left(\mathcal{B}_p\right)}\left\{I_u(\boldsymbol{u})=\frac{1}{2} \int_{\partial \mathcal{B}_p} \boldsymbol{T}[\boldsymbol{u}] \cdot \boldsymbol{u} \mathrm{d} S\right\}$, which we derive by associating a Lagrange multiplier $\lambda$ to the non-penetration constraint $g[\boldsymbol{u}] \geq 0$.  The Lagrangian of the problem is:

## Question: quadratic programming, dual problem?

$$
\mathcal{L}(\boldsymbol{u}, \lambda)=\frac{1}{2} \int_{\partial \mathcal{B}_p} \boldsymbol{T}[\boldsymbol{u}] \cdot \boldsymbol{u} \mathrm{d} S-\int_{\partial \mathcal{B}_p} \lambda\left(\boldsymbol{u} \cdot \boldsymbol{e}_3-h\right) \mathrm{d} S
$$

The Karush-Kuhn-Tucker optimality conditions imply stationarity of $\mathcal{L}$ with respect to $\boldsymbol{u}$ so that we can express $\lambda$ :

$$
\begin{aligned}
\frac{\partial \mathcal{L}}{\partial \boldsymbol{u}}=\mathbf{0} & \Leftrightarrow T[u]-\lambda \boldsymbol{e}_3=0 \\
& \Leftrightarrow \lambda=T[\boldsymbol{u}] \cdot \boldsymbol{e}_3 \\
& \Leftrightarrow \boldsymbol{u}=\mathcal{M}\left[\lambda \boldsymbol{e}_3\right] 
\end{aligned}
$$

$\lambda$ can be physically interpreted as the normal tractions acting on the surface. Replacing $\boldsymbol{u}=\mathcal{M}\left[p \boldsymbol{e}_3\right]$ in $\mathcal{L}$ (effectively computing the Legendre transform of $I_u$ ) leads to the quadratic program

$$
\inf _{p \boldsymbol{e}_3 \in \mathrm{SA}\left(\mathcal{B}_p\right)}\left\{I_p(p)=\frac{1}{2} \int_{\partial \mathcal{B}_p} p \boldsymbol{e}_3 \cdot \mathcal{M}\left[p \boldsymbol{e}_3\right] \mathrm{d} S-\int_{\partial \mathcal{B}_p} p h \mathrm{~d} S\right\}
$$

where the primary unknown $p$ is the normal surface traction and SA is the space of statically admissible tractions, i.e. tractions that satisfy $p \geq 0$ (for non-adhesive contact) and $\int_{\partial \mathcal{B}} p \mathrm{~d} S=$ $W$.

### Just not to be confused, $W$ is normal total load and we define $ \bar{p}=\frac{W}{S} $. 

In [36]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
import matplotlib.pyplot as plt

#define the parameters
W = 1e2  # total load
R = 1  # radius of demi-sphere
L = 2  # domain size
S = L**2  # domain area



#define material parameters
E = 1e3  # Young's modulus
nu = 0.3  # Poisson's ratio
E_star = E / (1 - nu**2)  # plane strain modulus 

#We generate a 2D coordinate space
n = 100
m = 100

x, y = np.meshgrid(np.linspace(0, L, n, endpoint=False), np.linspace(0, L, m, endpoint=False))#notice here that we use endpoint=False to avoid having the last point

x0 = 1
y0 = 1


In [37]:
# Based on the analytical solution, we can print the value of a and p0 as reference
a = (3*W*R/(4*E_star))**(1/3)
p0 = (6*W*E_star**2/(np.pi**3*R**2))**(1/3)

print(a, p0)

0.40866510127815564 285.8948173079934


## the separation function h depends on the contact model, since we just use a simple semi-sphere/flat surface contact, we keep it as $h(r)=-\frac{r^2}{2 R}$

In [38]:
# Define the separation h_matrix          #Since we are using the same model, we can use the same h function
h_matrix = - (x**2 + y**2)/(2*R)

# Initial pressure distribution
p_initial = np.full((n, m), W / S).flatten()  # Flatten for optimization

# Fourier transform of initial pressure distribution
P_fourier_flattened = np.fft.fft2(p_initial.reshape(n, m), norm='ortho').flatten()  # Keep flattened


## Minimization with FFT

First we use **scipy** library functions, *scipy.optimize.minimize* and *scipy.optimize.LinearConstraint* and FFT method with the nice property for convolution integral. We have the following convolution product:

$$
\mathcal{F}(u)= \mathcal{F}(f) \bullet \mathcal{F}(p) 
$$

We also need to define our Kernel function in fourrier domain:

$$
\mathcal{F}(f)=\frac{2}{E^* \sqrt{q_x^2+q_y^2}}
$$


In [39]:
#we define the frequency with q_x and q_y
q_x = 2 * np.pi * np.fft.fftfreq(n, d=L/n)
q_y = 2 * np.pi * np.fft.fftfreq(m, d=L/m)
QX, QY = np.meshgrid(q_x, q_y)

kernel_fourier = np.zeros_like(QX)  # Initialize the kernel array

## Possible to have negative value, because we set the *average value $C_0$* to be zero.

In [40]:
#non_zero_indices = (QX**2 + QY**2) != 0  # Avoid division by zero
kernel_fourier = 2 / (E_star * np.sqrt(QX**2 + QY**2))
kernel_fourier[0, 0] = 0  # Set the zero frequency component to zero

/var/folders/8h/9pns8j3d2sl62pk5rmmfs01h0000gn/T/ipykernel_6097/2267934034.py:2: RuntimeWarning: divide by zero encountered in divide
  kernel_fourier = 2 / (E_star * np.sqrt(QX**2 + QY**2))


From the instructions above, we have the formula for complementary energy:
$$
E_c\left(u_z, p\right)=\frac{1}{2} \int_s u_z \cdot p d s-\int_s p h d s
$$

In [41]:
#We call our complementary energy as cost function
def cost_function(P_flattened, kernel_fourier, h_matrix):
    # Ensure kernel_fourier and h_matrix are correctly shaped and prepared before this function is called
    
    # Reshape P_fourier_flattened back to its 2D shape to perform operations
    P_flattened = P_flattened.reshape((n, m))
    P_fourier = np.fft.fft2(P_flattened, norm='ortho')
    
    # Calculate u_z in the Fourier domain
    u_z_fourier = P_fourier * kernel_fourier
    
    # Transform u_z back to the spatial domain to compute the cost function
    u_z = np.fft.ifft2(u_z_fourier, norm='ortho').real
    
    # Calculate the first term of the cost function: 0.5 * sum(u_z * P), where P is also in the spatial domain
    term1 = 0.5 * np.sum(u_z * P_flattened)
    
    # Calculate the second term of the cost function: sum(P * h)
    term2 = np.sum(P_flattened * h_matrix)
    
    # The cost function is the difference of the two terms
    cost = term1 - term2
    return cost

We also have the formula for gap function:

$$
g[\boldsymbol{u}]:=\left.\boldsymbol{u} \cdot \boldsymbol{e}_3\right|_{\partial \mathcal{B}}-h
$$

In [42]:
# define the gradient of the cost function as jacobian
# the gradient of the cost function is actually the gap function
def gradient(P_flattened, kernel_fourier, h_matrix):
    # Ensure kernel_fourier and h_matrix are correctly shaped and prepared before this function is called
    
    # Reshape P_fourier_flattened back to its 2D shape to perform operations
    P_fourier = np.fft.fft2(P_flattened.reshape((n, m)), norm='ortho')
    
    # Calculate u_z in the Fourier domain
    u_z_fourier = P_fourier * kernel_fourier
    
    # Transform u_z back to the spatial domain to compute the cost function
    u_z = np.fft.ifft2(u_z_fourier, norm='ortho').real

    return (u_z - h_matrix).flatten()

In [43]:
# Flatten kernel_fourier and h_matrix if they are not already flattened, depending on how you pass them to the function
kernel_fourier_flattened = kernel_fourier.flatten()
h_matrix_flattened = h_matrix.flatten()


In [44]:
 # define the length of the pressure vector
length = len(P_fourier_flattened)  # p_bar is initial guess for the pressure
# first non_negative_constraint for the pressure
non_negative_constraint = LinearConstraint(np.eye(length), np.zeros(length), np.inf*np.ones(length))

# second avarage_constraint for the pressure
average_pressure_constraint = LinearConstraint(np.ones((1,length))/(length), W/S, W/S)

jac = lambda P_fourier_flattened: gradient(P_fourier_flattened, kernel_fourier, h_matrix)



result = minimize(lambda P_fourier_flattened: cost_function(P_fourier_flattened, kernel_fourier, h_matrix), P_fourier_flattened, method='trust-constr', jac=jac, constraints=[non_negative_constraint, average_pressure_constraint])


/Users/lizichen/anaconda3/lib/python3.11/site-packages/scipy/optimize/_trustregion_constr/minimize_trustregion_constr.py:315: ComplexWarning: Casting complex values to real discards the imaginary part
  x0 = np.atleast_1d(x0).astype(float)


In [ ]:
# Process the result
# .x refers to an attribute of the object result, which is returned by the minimize function from the scipy.optimize module in Python.
P_optimized_fourier = result.x.reshape((n, m))

displacement_optimized_fourier = P_optimized_fourier * kernel_fourier
displacement_optimized_real = np.fft.ifft2(displacement_optimized_fourier, norm='ortho').real

#print(displacement_optimized_real)

#plot P_optimized_fourier
plt.imshow(P_optimized_fourier, cmap='jet', origin='lower', extent=[0, L, 0, L])
print(P_optimized_fourier.mean())
plt.colorbar()
plt.show()

## We apply Hertz solution for comparasion:

we give our analytical normal displacement solution by Hertz

$$
\bar{u}_z=\frac{1-\nu^2}{E} \frac{\pi p_0}{4 a}\left(2 a^2-r^2\right), \quad r \leqslant a
$$


In [ ]:
F_value = 1e2 

#We keep other material parameters the same

# We define the distance from the center of the sphere
r = np.sqrt((x-x0)**2 + (y-y0)**2)

u_z = -(r**2)/(2*R)

# Correctly applying the displacement outside the contact area
outside_contact = r > a
u_z_outside = -(r[outside_contact]**2)/(2*R) + a * np.sqrt(r[outside_contact]**2 - a**2)/(np.pi*R) + (r[outside_contact]**2-2*a**2)*np.arccos(a/r[outside_contact])/(np.pi*R)
u_z[outside_contact] = u_z_outside

#plot u_z
plt.imshow(u_z, cmap='viridis', origin='lower', extent=[0, L, 0, L])
plt.colorbar(label='Displacement (u_z)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Displacement Field')
plt.show()

In [ ]:
#######################################################
## The following is comparasion of the two solutions
#######################################################



from mpl_toolkits.mplot3d import Axes3D

# Calculate the error between the real part of the displacement obtained through FFT and the analytical solution
error = np.abs(displacement_optimized_real - np.max(displacement_optimized_real) - u_z)

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Create a surface plot of the error
X, Y = np.meshgrid(np.linspace(0, L, n), np.linspace(0, L, m))
surf = ax.plot_surface(X, Y, error, cmap='viridis', edgecolor='none')

# Add a color bar which maps values to colors
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Labels and title
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Error')
ax.set_title('Error between Analytical and Fourier Method Displacement')

plt.show()

There is one more notation that needs to be clarified, $\mathcal{M}$ is a boundary integral operator defined in terms of fundamental solutions. Consequently, the integral operator $\mathcal{M}$ can be expressed as a convolution with respect to the horizontal coordinates. 

The gradient computation of the functional $I_p$ is straightforward with the use of $\mathcal{M}$:

$$
\nabla I_p(p)=\mathcal{M}\left[p \boldsymbol{e}_3\right] \cdot \boldsymbol{e}_3-h=u_3-h=g
$$


## Steepest descent algorithm

The approach proposed by Stanley and Kato(1997)[6] consists in taking steps in the $-\nabla I_p=-g$ direction then shifting and truncating the normal traction so that both the unilateral and equilibrium constraints are satisfied. Besides, its convergence rate is sub-optimal.



### Algorithm 1: Stanley and Kato (1997) steepest descent algorithm.

**Inputs:** normal total load  $W$, surface profile $\mathbf{H}$, tolerance  $\varepsilon$, maximum number of iterations  $N_{\text{max}}$.

1. Initialize $ P $ with the average constant load initial guess:
   $$ P \leftarrow \frac{W}{|\partial \mathbf{B}P|} $$
2. Set $ h_{\text{norm}} $ to the norm of  $\mathbf{H}$ :
   $$ h_{\text{norm}} \leftarrow \|\mathbf{H}\| $$
3. Set iteration counter $ k $ to zero:
   $$ k \leftarrow 0 $$

**Repeat until** $ e < \varepsilon $ **or** $ k > N_{\text{max}} $:

4. Calculate the gradient $ G $:
   $$ G \leftarrow M[\mathbf{P}e_3] \cdot e_3 - \mathbf{H} $$
5. Update $ P $ by subtracting $ G $:
   $$ P \leftarrow P - G $$
6. Find $ \alpha_0 $ that solves the following equation (where $ (\bullet)_+ $ is the ramp function):
   $$ \text{Find } \alpha_0 \text{ solution of } \Sigma (P + \alpha_0)_+ - W = 0 $$
7. Update $ P $ with $ \alpha_0 $:
   $$ P \leftarrow (P + \alpha_0)_+ $$
8. Calculate the error on complementarity $ e $:
   $$ e \leftarrow |P \cdot (G - \min(G))|/(Wh_{\text{norm}}) $$
9. Increment the iteration counter $ k $:
   $$ k \leftarrow k + 1 $$

**After exiting the loop**:

10. Ensure a positive gap by updating \( G \):
    $$ G \leftarrow G - \min(G) $$


In [ ]:
#same settings as before for material parameters and discretization

# Initial pressure distribution
P = np.full((n, m), W / S)  # Initial guess for the pressure

#same frequency components for fourier transform

# Initialize variables for the iteration
tol = 1e-6  # Tolerance for convergence
iter_max = 10000  # Maximum number of iterations
k = 0  # Iteration counter
error = np.inf  # Initialize error

As we can see, in the solution loop, we need a root-finding function for $\alpha_0$

In [ ]:
def find_alpha_0(P, W, alpha_l, alpha_r, tol):
    # approximates a root, R, of f bounded 
    # by a and b to within tolerance 
    # | f(m) | < tol with m the midpoint 
    # between a and b Recursive implementation
    
    
    def f(alpha):
        return np.mean(P + alpha) - W/S #since we give mean function, we should apply the avarge loading
    
    # check if a and b bound a root
    while np.sign(f(alpha_l)) == np.sign(f(alpha_r)):
        alpha_r *= 2
        #raise Exception(f"The scalars alpha_l and alpha_r do not bound a root {np.sign(f(alpha_l))} {np.sign(f(alpha_r))}")
    # get midpoint
    alpha_c = (alpha_l + alpha_r)/2
    

    if np.abs(f(alpha_c)) < tol:
        # stopping condition, report alpha_c as root
        return alpha_c
    elif np.sign(f(alpha_l)) == np.sign(f(alpha_c)):
        # case where m is an improvement on a. 
        # Make recursive call with a = m
        return find_alpha_0(P, W, alpha_c, alpha_r, tol)
    elif np.sign(f(alpha_r)) == np.sign(f(alpha_c)):
        # case where m is an improvement on b. 
        # Make recursive call with b = m
        return find_alpha_0(P, W, alpha_l, alpha_c, tol)


In [ ]:
while np.abs(error) > tol and k < iter_max:
    # Calculate the gradient G in the Fourier domain and transform it back to the spatial domain
    P_fourier = np.fft.fft2(P, norm='ortho')
    G_fourier = P_fourier * kernel_fourier
    G = np.fft.ifft2(G_fourier, norm='ortho').real - h_matrix
    
    # Update P by subtracting G
    P = P - G
    
    # Ensure P is non-negative
    P = np.maximum(P, 0)
    
    # Adjust P to satisfy the total load constraint
    alpha_0 = find_alpha_0(P, W/S, -np.max(P), W, tol)
    #alpha_0 = find_alpha_0(P, W, -1e2, 1e2, 1e-6)
    P += alpha_0
    P[P < 0] = 0
    
    # Calculate the error for convergence checking
    error = np.vdot(P, (G - np.min(G))) / (P.size*W) #/ np.linalg.norm(h_matrix)
    print(error, k)
    
    k += 1  # Increment the iteration counter


# Ensure a positive gap by updating G
G = G - np.min(G)

In [ ]:
displacement_fourier = P_fourier * kernel_fourier
displacement = np.fft.ifft2(displacement_fourier, norm='ortho').real

plt.imshow(P, cmap='jet', origin='lower', extent=[0, L, 0, L])
plt.colorbar(label='Pressure (P)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Pressure Field')
plt.show()

we give our analytical normal displacement solution by Hertz

$$
\bar{u}_z=\frac{1-\nu^2}{E} \frac{\pi p_0}{4 a}\left(2 a^2-r^2\right), \quad r \leqslant a
$$


## The Hertz solution has been computed before, the comparasion is given as:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Calculate the error between the real part of the displacement obtained through FFT and the analytical solution
error = np.abs(displacement - np.max(displacement) - u_z)

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Create a surface plot of the error
X, Y = np.meshgrid(np.linspace(0, L, n), np.linspace(0, L, m))
surf = ax.plot_surface(X, Y, error, cmap='viridis', edgecolor='none')

# Add a color bar which maps values to colors
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Labels and title
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Error')
ax.set_title('Error between Analytical and Fourier Method Displacement')

plt.show()

## Conjugate gradient algorithm

The method proposed by Polonsky and Keer (1999b)[7] is a variation of a conjugate gradient algorithm with an active set to accelerate the convergence. As expected from numerical linear algebra, it outperforms the steepest descent.


## Comparasion with other online solver

## Reference:

[1] Frérot, Lucas Henri Galilée. ‘Bridging Scales in Wear Modeling with Volume Integral Methods for Elastic-Plastic Contact’, n.d.

[2] Yastrebov, Vladislav A. Numerical Methods in Contact Mechanics. Numerical Methods in Engineering Series. Hoboken, NJ: Wiley, 2013.

[3] Kalker, J. J. ‘Variational Principles of Contact Elastostatics’. IMA Journal of Applied Mathematics 20, no. 2 (1977): 199–219. https://doi.org/10.1093/imamat/20.2.199.

[4] Vladislav A. Yastrebov, Contact mechanics and elements of tribology, Lecture 4.b, Contact and transport at small scales, Open Course Contact Mechanics and Elements of Tribology, January 23, 2024, https://cmet.yastrebov.fr/index.html

[5] Rey, Valentine, Guillaume Anciaux, and Jean-François Molinari. ‘Normal Adhesive Contact on Rough Surfaces: Efficient Algorithm for FFT-Based BEM Resolution’. Computational Mechanics 60, no. 1 (July 2017): 69–81. https://doi.org/10.1007/s00466-017-1392-5.

[6] Stanley, H. M., and T. Kato. ‘An FFT-Based Method for Rough Surface Contact’. Journal of Tribology 119, no. 3 (1 July 1997): 481–85. https://doi.org/10.1115/1.2833523.

[7] Polonsky, I.A., and L.M. Keer. ‘A Numerical Method for Solving Rough Contact Problems Based on the Multi-Level Multi-Summation and Conjugate Gradient Techniques’. Wear 231, no. 2 (July 1999): 206–19. https://doi.org/10.1016/S0043-1648(99)00113-1.

[8] https://gitlab.com/tamaas/tamaas/-/blob/master/examples/scipy_penalty.py?ref_type=heads

[9] https://www.pecms.cn/